# AWS redshift columnar storage
https://docs.aws.amazon.com/redshift/latest/dg/c_columnar_storage_disk_mem_mgmnt.html


---

# 🔎 Visual Step-by-Step Demo

## Matrix Example

$$
A =
\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}
$$

---

## 🟦 Fetching **Column 2**

### Row-Based (OLTP style): scan across rows

```
Row 1: [1,  2,  3]   → take  2
Row 2: [4,  5,  6]   → take  5
Row 3: [7,  8,  9]   → take  8
                       --------
Column 2 = [2, 5, 8]^T
```

➡ Must touch **every row**.

---

### Column-Based (OLAP style): direct column access

```
C2 = [2, 5, 8]^T   (already stored together!)
```

➡ Just one **contiguous read**.

---

## 🟥 Fetching **Row 1**

### Row-Based (OLTP style): direct

```
R1 = [1, 2, 3]   (already stored together!)
```

➡ Super fast.

---

### Column-Based (OLAP style): assemble from each column

```
From C1 → take 1
From C2 → take 2
From C3 → take 3
           --------
Row 1 = [1, 2, 3]
```

➡ Must touch **every column**.

---

# ✅ Summary Visual

| Operation    | Row Storage     | Column Storage              |
| ------------ | --------------- | --------------------------- |
| Fetch row    | **Direct** ✅    | Assemble from all columns ❌ |
| Fetch column | Scan all rows ❌ | **Direct** ✅                |

---





#### **1. Data Generation**

Instead of writing a few rows manually, show learners how to **auto-generate dummy data**. This avoids typing fatigue and makes benchmarking meaningful with 10k+ records.

```python
# Generate row-based data
row_data = [
    {"id": i, "name": f"User{i}", "age": 18 + i % 50, "score": 50 + (i * 3) % 50}
    for i in range(1, 100001)
]

# Generate column-based data
column_data = {
    "id": [i for i in range(1, 100001)],
    "name": [f"User{i}" for i in range(1, 100001)],
    "age": [18 + i % 50 for i in range(1, 100001)],
    "score": [50 + (i * 3) % 50 for i in range(1, 100001)]
}
```

💡 Now your benchmarks will show clearer differences.

---

#### **2. Benchmarking with Multiple Runs**

Timing a single fetch can be too fast to measure. Use **`timeit`** or loop over operations for more realistic timing.

```python
import time

# row_data Test: Fetch 10,000 full rows
start = time.time()
by_rows = [row["score"] for row in row_data]
by_rows_time = time.time() - start

# column_data Test: Fetch 10,000 full rows
start = time.time()
by_column = column_data["score"]
by_column_time = time.time() - start


# row_data Test: Compute row avg
start = time.time()
by_rows = [row["score"] for row in row_data]
row_avg_score = sum(by_rows) / len(by_rows)
row_avg_time = time.time() - start


# column_data Test: Compute column avg
start = time.time()
column_avg_score = sum(column_data["score"]) / len(column_data["score"])
column_avg_time = time.time() - start

print(f"Row fetch all scores: {by_rows_time:.5f}s")
print(f"Column fetch all scores: {by_column_time:.5f}s")
print(f"Row avg score: {row_avg_time:.5f}s")
print(f"Column avg score: {column_avg_time:.5f}s")
```

---

#### **3. Memory Estimation**

Your string-based estimate works, but for clarity, you can also show **`sys.getsizeof`** so students see Python’s actual object overhead:

```python
import sys

sys_row_mem = sum(sys.getsizeof(row) for row in row_data)
sys_column_mem = sum(sys.getsizeof(vals) for vals in column_data.values())

print(f"Row storage: ~{sys_row_mem/(1024**2):,.2f} MB")
print(f"Column storage: ~{sys_column_mem/(1024**2):,.2f} MB")

```

---

#### **4. Discussion Prompts**

At the end, give 2–3 **questions for learners**:

* Why did row-based fetching feel faster for individual records?
* Why did column-based storage excel at aggregations?
* Can you imagine a real-world system that benefits from each?

---

✅ With these tweaks, your demo will:

* Scale better (10k records = meaningful benchmarks).
* Show **clear, measurable differences**.
* Be **hands-on and engaging**, not just theoretical.



# Postgresql and clickhouse columnar storage


## clickhouse
http://172.21.8.40:5521/
### config:
    http://localhost:8123
    admin
    password

## pgamin
http://172.21.8.40:5050
### config:
    admin@example.com
    admin
